In [2]:
!python setup_04.py

Running command: sudo apt-get update
Command output: Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:4 https://deb.nodesource.com/node_16.x focal InRelease [4583 B]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:10 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1334 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [31.2 kB]
Get:13 http://archive.

In [4]:
    import os
    import pytesseract
    from PIL import Image
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
    from sklearn.metrics import confusion_matrix, classification_report
    import pandas as pd
    from datasets import load_dataset

In [6]:
ocr_dataset = load_dataset("vaclavpechtor/rvl_cdip-small-200")

Using custom data configuration vaclavpechtor--rvl_cdip-small-200-346d6eed8e7dc106


Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset imagefolder downloaded and prepared to /root/.cache/huggingface/datasets/vaclavpechtor___imagefolder/vaclavpechtor--rvl_cdip-small-200-346d6eed8e7dc106/0.0.0/48efdc62d40223daee675ca093d163bcb6cb0b7d7f93eb25aebf5edca72dc597. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
import pickle

def save_dataset(dataset, filename):
    with open(filename, 'wb') as f:
        pickle.dump(dataset, f)

In [13]:
def load_dataset(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [15]:
import os

if os.path.exists('./dataset/rvl-cdip-small-200/ocr_dataset.pkl'):
    ocr_dataset = load_dataset('./dataset/rvl-cdip-small-200/ocr_dataset.pkl')
else:
    ocr_dataset = ocr_dataset.map(lambda x: {"text": ocr_image(x["image"])})
    save_dataset(ocr_dataset, './dataset/rvl-cdip-small-200/ocr_dataset.pkl')

In [16]:
ocr_dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label', 'text'],
        num_rows: 2560
    })
})

In [17]:
ocr_dataset.set_format(type='pandas')

In [18]:
df = ocr_dataset['train'][:]

In [19]:
df.head()

,image,label,text
0,<PIL.TiffImagePlugin.TiffImageFile image mode=...,0,"\n\na\n\nCevetrom Phi""eMonis,\n\n” Saratoe\n\..."
1,<PIL.TiffImagePlugin.TiffImageFile image mode=...,0,"eae Arizona\nFebruary 20-21, 1984 oo\n\nSit 00..."
2,<PIL.TiffImagePlugin.TiffImageFile image mode=...,0,ae\nPRIVEE SITET\neae beaters\n\n \n
3,<PIL.TiffImagePlugin.TiffImageFile image mode=...,0,a\nholds backtar;\nf ut lets Ue Sul.\nmenthol ...
4,<PIL.TiffImagePlugin.TiffImageFile image mode=...,0,fl .\na\n\n \n\n \n\nyou should know that many...
